# Data Preprocessing

This python notebook contains code needed to extract data for analysis from several source files containing personally identifiable information (PII). The outputs are data files in `data/main` that contain only data values for analysis and participant IDs, but no PII.

In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path
Path("../data/main").mkdir(parents=True, exist_ok=True)

# Signups and informed consent

We have one manually maintained spreadsheet for participant contact information.

In [ ]:
def clean_id_string(series):
    return series.str.upper().str.strip()

In [ ]:
import secrets 

import io
import msoffcrypto

decrypted_workbook = io.BytesIO()
with open('../data/RAW/consent_protected.xlsx', 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=secrets.excel_pw)
    office_file.decrypt(decrypted_workbook)

consent = pd.read_excel(
    decrypted_workbook,
    sheet_name='consent',
    usecols=[
        'id',
        'id_family',
        'diet_reported',
        'daycare',
        'daycare_group',
        'daycare_municipal',
        'discontinued',
        'consent_date',
        'daycare_municipal',
        'dob'
    ]
)
consent.id = clean_id_string(consent.id)
consent.id_family = clean_id_string(consent.id_family)


reserch_steps = pd.read_excel(
    decrypted_workbook,
    sheet_name='reserch_steps',
    usecols=[
        'id',
        'date_blood',
        'urine_sample',
        'fecal_sample',
    ]
)
reserch_steps.id = clean_id_string(reserch_steps.id)

Generating pseudonymzation id's for daycare centers and groups within.

Using an alphabetized list of names to keep generated id mappings fairly stable. They will only change if new inputs are made manually to the source file, which is not expected.

Id stability should not be necessary, though. Analyses using the generated master only need these for grouping. They should not depend on ids being the same in subsequent version of the file.

Daycare centers have unique names. Groups are made globally unique by concatenating center and group name.

In [ ]:
keys = sorted(consent.daycare.dropna().unique())

dc_id_map = dict(zip(keys,range(len(keys))))

consent['dc_id'] = consent.daycare.map(dc_id_map)

In [ ]:
dc_dcg = consent.daycare + consent.daycare_group

keys = sorted(dc_dcg.dropna().unique())

dc_dcg_id_map = dict(zip(keys,range(len(keys))))

consent['dc_group_id'] = dc_dcg.map(dc_dcg_id_map)

In [ ]:
#converting to bool column
consent.daycare_municipal = consent.daycare_municipal==1

In [ ]:
#exporting id mapping for later verification

with pd.ExcelWriter('../data/RAW/daycare_id_mapping.xlsx') as writer:
    pd.DataFrame.from_dict(
        dc_id_map,
        orient='index'
    ).to_excel(
        writer, 
        sheet_name='dc_id'
    )
    
    pd.DataFrame.from_dict(
        dc_dcg_id_map,
        orient='index'
    ).to_excel(
        writer, 
        sheet_name='dc_group_id'
    )

# Redcap questionnaire

Participants filled in background questionnaires in RedCAP, one for each child participant and one for caregivers participating in food records and biometric sampling.

## Children

In [ ]:
#reading only specific fields from redcap data to avoid PII
#using redcap record_id as pandas index
rc_c = pd.read_csv(
    '../data/RAW/redcap/child.csv',
    usecols=lambda c: c in ['record_id','id_child','id_family','mira2_lapsen_taustatieto_ja_ruoankyttkysely_timestamp','sex'] or c.startswith('diet_'),
    index_col=0
)
rc_c.id_child = clean_id_string(rc_c.id_child)
rc_c.id_family = clean_id_string(rc_c.id_family)


In [ ]:
s = rc_c.mira2_lapsen_taustatieto_ja_ruoankyttkysely_timestamp
rc_c['date_q'] = pd.to_datetime(s.mask(s == '[not completed]'))

In [ ]:
#helper function to update value of given column on row indexed by redcap record_id
def set_col_for_record(record_id, col, val):
    
    #at points to a single cell by index and col name
    old_val = rc_c.at[record_id,col]
    idf = rc_c.at[record_id,'id_family']
    idc = rc_c.at[record_id,'id_child']
    
    rc_c.at[record_id,col] = val

    print(f'{record_id}, {idf}, {idc}, {col}: {old_val} --> {val}')

In [ ]:
# M3339 shows as not completed in form data, but data is valid and input date confirmed directly from participant

set_col_for_record(
    70,
    'date_q',
    '2022-06-21'
)


In [ ]:
#dropping incomplete answers
rc_c = rc_c[rc_c.date_q.notna()]

In [ ]:
#fixing known input errors

#capitalizing all ids
rc_c.id_family = rc_c.id_family.str.capitalize()
rc_c.id_child = rc_c.id_child.str.capitalize()

In [ ]:
# helper functions to define valid IDs and returning invalid rows

def is_valid_id(s):
    return isinstance(s,str) and len(s)==5 and s[-4:].isdigit()

def is_valid_id_family(s):
    return is_valid_id(s) and s.startswith('P')

def is_valid_id_child(s):
    return is_valid_id(s) and s.startswith('M')

def erroneous_ids():
    valid_id_family = rc_c.id_family.apply(is_valid_id_family)
    valid_id_child = rc_c.id_child.apply(is_valid_id_child)
    
    return rc_c.loc[
        rc_c.date_q.notna() & (
            ~valid_id_family | ~valid_id_child
        )
    ][['id_family','id_child']]
    

In [ ]:
erroneous_ids()

In [ ]:
#P was missing from family_id for M3309
set_col_for_record(
    65,
    'id_family',
    'P1001'
)

In [ ]:
#P was missing from family_id for M3310
set_col_for_record(
    66,
    'id_family',
    'P1001'
)

In [ ]:
#P was replaced with zero in family_id for M3313
set_col_for_record(
    57,
    'id_family',
    'P1002'
)

In [ ]:
#family_id incorrect M3400
set_col_for_record(
    78,
    'id_family',
    'P1030'
)

In [ ]:
#family_id had i for 1 M3416
set_col_for_record(
    13,
    'id_family',
    'P1035'
)

In [ ]:
#family_id was M3495
set_col_for_record(
    37,
    'id_family',
    'P1060'
)

In [ ]:
#family_id was M3501
set_col_for_record(
    11,
    'id_family',
    'P1062'
)

In [ ]:
#swapping values where family- and child ids are swapped
m = rc_c.id_family.str.startswith('M') & rc_c.id_child.str.startswith('P')

rc_c.loc[m, ['id_family', 'id_child']] = (
    rc_c.loc[m, ['id_child', 'id_family']].values)

In [ ]:
erroneous_ids()

# Diet classification of participants

Study participants have filled in RedCap survey forms that differ for children and their caretakers. On both forms there are multiple choice questions to more clearly specify the diet participants adhere to.

This code contains all interpretations of survey answers to group participants to diet categories.


## Form for Children
Same row of check boxes was presented for both home and daycare diets:


- Ei erityisruokavaliota tai välttämisruokavaliota
- Laktoositon tai vähälaktoosinen ruokavalio
- Gluteeniton ruokavalio (vältetään vehnää,ruista ja ohraa)
- Ruokavalio, joka ei sisällä punaista lihaa
- Vegaaninen ruokavalio (ei sisällä mitään eläinperäisiä tuotteita)
- Kasvisruokavalio, joka sisältää yhtä tai useampaa seuraavista eläinkunnan tuotteista: kalaa, kananmunaa ja/tai maitotuotteita
- Ruokarajoituksia uskonnollisista syistä
- Muu ruokavalio

The code below encodes selections as `1`s and unchecked boxes as `0`s. For each form, we get an eight-character string of `0`s and `1`s. All combinations found are mapped to a diet category according to the research group's interpretation.

In [ ]:
diet_id = {
    '00001000': 0,
    '00000100': 1,
    '00001100': 1, #contradictory answer
    '10000100': 1, #contradictory answer
    '01000100': 2,
    '00010100': 3,
    '01000000': 4,
    '10000001': 5,
    '10000000': 6,
}
diet_name = {    
    0: 'vegan',
    1: 'vegetarian',
    2: 'vegetarian lactose-free',
    3: 'vegetarian no red meat', 
    4: 'mixed diet lactose-free',
    5: 'mixed diet other diet', 
    6: 'mixed diet',
}
diet_main = {    
    0: 'vegan',
    1: 'vegetarian',
    2: 'vegetarian',
    3: 'vegetarian', 
    4: 'mixed_diet',
    5: 'mixed_diet', 
    6: 'mixed_diet',
}

In [ ]:
rc_c['diet_dc_ticks'] = \
rc_c.diet_dc___1.astype(str) + \
rc_c.diet_dc___2.astype(str) + \
rc_c.diet_dc___3.astype(str) + \
rc_c.diet_dc___4.astype(str) + \
rc_c.diet_dc___5.astype(str) + \
rc_c.diet_dc___6.astype(str) + \
rc_c.diet_dc___7.astype(str) + \
rc_c.diet_dc___8.astype(str)

rc_c['diet_home_ticks'] = \
rc_c.diet_home___1.astype(str) + \
rc_c.diet_home___2.astype(str) + \
rc_c.diet_home___3.astype(str) + \
rc_c.diet_home___4.astype(str) + \
rc_c.diet_home___5.astype(str) + \
rc_c.diet_home___6.astype(str) + \
rc_c.diet_home___7.astype(str) + \
rc_c.diet_home___8.astype(str)

rc_c['diet_dc_id'] = rc_c.diet_dc_ticks.map(diet_id)
rc_c['diet_dc'] = rc_c.diet_dc_id.map(diet_name)

rc_c['diet_home_id'] = rc_c.diet_home_ticks.map(diet_id)
rc_c['diet_home'] = rc_c.diet_home_id.map(diet_name)

rc_c['diet_main'] = rc_c[['diet_dc_id', 'diet_home_id']].max(axis=1).map(diet_main)

## Caregivers

This data not used yet.

# Samples

Participants gave fecal and urine samples for lab testing. Age at sample time is calculated for analysis and to hide DoB. Existence of date is used to track which participants gave samples.

In [ ]:
f_samples = pd.read_excel(
    '../data/RAW/fecal_and_urine_samples.xlsx',
    sheet_name='feces',
    usecols=[
        'id',
        'fecal_date',
    ]
)

In [ ]:
u_samples = pd.read_excel(
    '../data/RAW/fecal_and_urine_samples.xlsx',
    sheet_name='urines',
    usecols=[
        'id',
        'urine_date',
    ]
)

In [ ]:
#time inputs in spreadsheet are all kinds of off
from datetime import datetime, time
from math import isnan

def clean_time(t):
    if isinstance(t, time):
        #correct inputs are parsed when reading excel, keeping those
        return t
    if isinstance(t, float):
        #empty cells come in as NaN
        if isnan(t):
            return None
        #short inputs get parsed as float
        t = str(t)
    if isinstance(t, str):
        #remaining problems should be strings
        #removing any spaces and replacing . with : 
        t = t.strip(' .').replace(' ','').replace('.',':')
        if t.count(':')<2:
            #in some cases seconds are missing, so adding 00
            t += ':00'
        return datetime.strptime(t, "%H:%M:%S").time()
    raise Exception(f'Not prepared to process {type(t)}')

b_samples = pd.read_excel(
    '../data/RAW/blood_samples.xlsx',
    sheet_name='blood_sample_collection',
    usecols=[
        'id',
        'blood_collected_time',
    ],
)

b_samples['blood_time_of_day'] = b_samples.blood_collected_time.apply(clean_time)

# Merging

Merging dataframes in participant ID.

For privacy, no date columns are written out to master, age at sample times is calculated.

In [ ]:
participants = consent.merge(
    reserch_steps,
    left_on=['id'],
    right_on=['id'],
    how='outer'
).merge(
    rc_c,
    left_on=['id'],
    right_on=['id_child'],
    how='outer',
    suffixes=('_consent', '_redcap')
).merge(
    f_samples,
    on='id',
    how='outer',
).merge(
    u_samples,
    on='id',
    how='outer',
).merge(
    b_samples,
    on='id',
    how='outer',
)

In [ ]:
participants['age_q'] = (participants.date_q.dt.normalize() - participants.dob).dt.days

In [ ]:
participants['age_informed_consent'] = (participants.consent_date - participants.dob).dt.days

In [ ]:
participants['age_blood'] = (pd.to_datetime(participants.date_blood, errors='coerce') - participants.dob).dt.days

In [ ]:
participants['age_feces'] = (participants.fecal_date - participants.dob).dt.days

In [ ]:
participants['age_urine'] = (participants.urine_date - participants.dob).dt.days

## ID Checks

In [ ]:
#checking for mismatched family ids
participants[participants.id_family_redcap.notna() & (participants.id_family_consent != participants.id_family_redcap)][['id','id_family_consent','id_family_redcap']]

In [ ]:
#number of family ids missing in redcap data 
participants.id_family_redcap.isna().sum()

In [ ]:
#using family_id from consent data
participants['id_family'] = participants.id_family_consent

# Participants master file

Combining person-level data to a single table without PII.

For dta hygiene, only explicitly selected columns are written out to the master data file on `data/main`.

In [ ]:
variables= [
    'id',
    'id_family',
    'age_informed_consent',
    'diet_reported',
    'daycare_municipal',
    'discontinued',
    'dc_id',
    'dc_group_id',
    'age_blood',
    'blood_time_of_day',
    'age_feces',
    'age_urine',
    'sex',
    'age_q',
    'diet_dc_id',
    'diet_dc',
    'diet_home_id',
    'diet_home',
    'diet_main',
]

descriptions = [
    'unique identifier of study participant',
    'unique identifier participating family',
    'age of participant at time reserch group received informed consent',
    'diet reported at enrollment',
    'boolean denoting whether daycare center attended is municipally operated',
    'boolean denoting whether participant discontinued the study before completion',
    'unique identifier of daycare center',
    'unique identifier of group at daycare center',
    'age of participant at time of blood sampling',
    'time of day when blood sample was taken',
    'age of participant at time of fecal sampling',
    'age of participant at time of urine sampling',
    'sex of participant (1=F, 2=M)',
    'age of participant at time of completing questionnaire',
    'unique identifier for diet subgroup at daycare based on questionnaire',
    'label for diet group at daycare based on questionnaire',
    'unique identifier for diet subgroup at home based on questionnaire',
    'label for diet group at home based on questionnaire',
    'label for overall diet group',
]

In [ ]:
with pd.ExcelWriter('../data/main/participants_master.xlsx') as writer:
    participants[variables].to_excel(
        writer, 
        sheet_name='participants',
        index=False
    )
    
    pd.DataFrame(
        {'variable_name': variables,
         'variable_description': descriptions,
        }
    ).to_excel(
        writer, 
        sheet_name='dictionary',
        index=False
    )